# Biome Level statistics for model: HM1

In [1]:
# import the libraries
import ee
import pandas as pd
import os
import numpy as np
import random
from random import sample
import itertools 
import geopandas as gpd
from sklearn.metrics import r2_score
from termcolor import colored # this is allocate colour and fonts type for the print title and text
from IPython.display import display, HTML

In [2]:
#set the working directory of local drive for Grid search result table loading
# os.getcwd()

In [3]:
# initialize the earth engine API
ee.Initialize()

## 1 Load the required composites and images

In [4]:
# load the basic maps that needed for the analysis
# load the carbon concentration map
carbonConcentration = ee.Image("users/leonidmoore/ForestBiomass/Biome_level_Wood_Carbon_Conentration_Map")
# load the two composites tha will be used in the analysis
compositeImage =ee.Image("users/leonidmoore/ForestBiomass/20200915_Forest_Biomass_Predictors_Image")
compositeImageNew = ee.Image("projects/crowtherlab/Composite/CrowtherLab_Composite_30ArcSec")
# load the biome layer 
biomeLayer = compositeImage.select("WWF_Biome")
# define a pixel area layer with unit km2
pixelAreaMap = ee.Image.pixelArea().divide(10000);
# define the boundary geography reference
unboundedGeo = ee.Geometry.Polygon([-180, 88, 0, 88, 180, 88, 180, -88, 0, -88, -180, -88], None, False)

## 2 Load the biomass density maps

In [5]:
# load the carbon density layers
potentialDensity = ee.Image("users/leonidmoore/ForestBiomass/GroundSourcedModel/EnsambledMaps/Predicted_HM1_Potential_density_Ensambled_Mean").unmask()
presentDensity =  ee.Image("users/leonidmoore/ForestBiomass/SpawnMap/Spawn_Harmonized_AGB_density_Map_1km").select('agb').unmask()

# define the standard projection
stdProj = potentialDensity.projection();

## 3 Adjust the present and potential density maps

In [6]:
# load the present and potential forest cover
presentForestCover = compositeImage.select('PresentTreeCover').unmask() # uniform with potential in the  0-1 scale
potentialCoverAdjusted = ee.Image("users/leonidmoore/ForestBiomass/Bastin_et_al_2019_Potential_Forest_Cover_Adjusted").unmask().rename('PotentialForestCover')
# define the present and potential forest cover masks
presentMask= presentForestCover.gt(0)
potentialMask= potentialCoverAdjusted.gt(0)

# check the difference of the two density maps
potentialHigher = potentialDensity.multiply(pixelAreaMap).subtract(presentDensity.multiply(pixelAreaMap)).gte(0)
potentialLower = potentialDensity.multiply(pixelAreaMap).subtract(presentDensity.multiply(pixelAreaMap)).lt(0)
# replace the lower potential value by present biomass density value
agbPotentialDensity = presentDensity.multiply(potentialLower).add(potentialDensity.multiply(potentialHigher))

agbPresentDensity = presentDensity

## 4 Partioning the potential cover into different landuse types

In [7]:
# Load all the landuse type layers
croplandOrg = ee.Image("users/leonidmoore/ForestBiomass/HYDE31/cropland_Percent").rename('cropland').divide(100).reproject(crs=stdProj);
grazingOrg = ee.Image("users/leonidmoore/ForestBiomass/HYDE31/grazing_Percent").rename('grazing').divide(100).reproject(crs=stdProj);
pastureOrg = ee.Image("users/leonidmoore/ForestBiomass/HYDE31/pasture_Percent").rename('pasture').divide(100).reproject(crs=stdProj);
rangelandOrg = ee.Image("users/leonidmoore/ForestBiomass/HYDE31/rangeland_Percent").rename('rangeland').divide(100).reproject(crs=stdProj);
urbanOrg = compositeImage.select(['LandCoverClass_Urban_Builtup']).divide(100).unmask().reproject(crs=stdProj);
snowIceOrg = compositeImageNew.select(['ConsensusLandCoverClass_Snow_Ice']).divide(100).unmask().reproject(crs=stdProj);
openWaterOrg = compositeImageNew.select(['ConsensusLandCoverClass_Open_Water']).divide(100).unmask().reproject(crs=stdProj);
# define the total landcover types
sumCover = presentForestCover.add(pastureOrg).add(rangelandOrg).add(croplandOrg).add(urbanOrg).add(openWaterOrg).add(snowIceOrg);
oneSubtract = ee.Image(1).subtract(sumCover);
freeland = oneSubtract.multiply(oneSubtract.gte(0));
# get the scale ratio for pixels with sumCover larger than 1
scaleRatio = ee.Image(1).subtract(presentForestCover).divide(sumCover.subtract(presentForestCover)).multiply(oneSubtract.lt(0));
# get the ratio of these three disturbed maps
pasture = pastureOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(pastureOrg.multiply(oneSubtract.gte(0))).unmask();
rangeland = rangelandOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(rangelandOrg.multiply(oneSubtract.gte(0))).unmask();
cropland = croplandOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(croplandOrg.multiply(oneSubtract.gte(0))).unmask();
urban = urbanOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(urbanOrg.multiply(oneSubtract.gte(0))).unmask();
openWater = openWaterOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(openWaterOrg.multiply(oneSubtract.gte(0))).unmask();
snowIce = snowIceOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(snowIceOrg.multiply(oneSubtract.gte(0))).unmask();
sumTT = presentForestCover.add(pasture).add(rangeland).add(cropland).add(urban).add(freeland).add(openWater).add(snowIce).unmask();

effectivePotentialMask = freeland.add(rangeland).add(pasture).add(cropland).add(urban).gt(0);
# there are some pixels without any landcover survived but with open water and ice and snow. here we mask these pixels out
sumlandCover = pastureOrg.add(rangelandOrg).add(croplandOrg).add(urbanOrg).add(freeland);
restorationMap = potentialCoverAdjusted.subtract(presentForestCover).mask(effectivePotentialMask).unmask();

# sum all these scaled layersv
scaledSum = pasture.add(rangeland).add(cropland).add(urban).add(freeland);
potentialCoverFinal = restorationMap.add(presentForestCover);
# allocate the potential equally to each layer
freelandPotentialCover = freeland.divide(scaledSum).multiply(restorationMap).unmask();
rangelandPotentialCover = rangeland.divide(scaledSum).multiply(restorationMap).unmask();
pasturePotentialCover = pasture.divide(scaledSum).multiply(restorationMap).unmask();
croplandPotentialCover = cropland.divide(scaledSum).multiply(restorationMap).unmask();
urbanPotentialCover = urban.divide(scaledSum).multiply(restorationMap).unmask();
#  allocate the freeland potential in pixels with forest cover larger than 10% to conservation potential
freelandForConsevation = freelandPotentialCover.multiply(presentForestCover.gte(0.1)).unmask();
maximumPotentialCover = freelandForConsevation.add(presentForestCover);
# calucate the reall freeland outside of forest
freelandLeftMap = freelandPotentialCover.subtract(freelandForConsevation).unmask()# the left positive pixels are real freeland pixels

## 5 Partioning the biomass potential into different landuse types

In [8]:
# calculate the existing carbon, present potential carbon and absolute potential carbon in forests
absoluteImage1 = agbPresentDensity.multiply(pixelAreaMap).multiply(presentMask).divide(1000000000).rename('Present')
absoluteImage3 = agbPotentialDensity.multiply(pixelAreaMap).multiply(potentialCoverFinal.gt(0)).divide(1000000000).rename('AbsolutePotential')
# get the sum of the potential covers
potentialCoverSum = freelandLeftMap.add(rangelandPotentialCover).add(pasturePotentialCover).add(croplandPotentialCover).add(urbanPotentialCover)

trueRestorationPotential = absoluteImage3.subtract(absoluteImage1).multiply(1000000000)
ratioPotentialBiomassDensity = absoluteImage1.multiply(potentialCoverFinal.divide(presentForestCover))
#  get the real for conservation potential
realDensityIncreased = absoluteImage3.subtract(absoluteImage1).mask(absoluteImage3.subtract(ratioPotentialBiomassDensity).gt(0)).unmask()
realDensityNotIncreased = absoluteImage3.subtract(absoluteImage1).mask(absoluteImage3.subtract(ratioPotentialBiomassDensity).lte(0)).unmask()
trueReforestationPotential = realDensityNotIncreased.add(realDensityIncreased.multiply(ee.Image(1).subtract(presentForestCover.divide(potentialCoverFinal))))

conservationPotentialPart1 = realDensityIncreased.multiply(presentForestCover.add(freelandForConsevation).divide(potentialCoverFinal))
conservationPotentialPart2 = realDensityNotIncreased.multiply(freelandForConsevation.divide(potentialCoverFinal.subtract(presentForestCover)))

# calculate the part of the potential inside the forest cover which was allocate to conservation potential.
freelandForConservation1 = realDensityIncreased.multiply(freelandForConsevation.divide(potentialCoverFinal))
freelandForConservation = freelandForConservation1.add(conservationPotentialPart2).rename('FreeToConservation')


absoluteImage2 = conservationPotentialPart1.add(conservationPotentialPart2).add(absoluteImage1).rename('PresentPotential')

trueReforestationPotential = absoluteImage3.subtract(absoluteImage2)

absoluteImage4 = trueReforestationPotential.multiply(freelandLeftMap.divide(potentialCoverSum)).rename('FreelandPotential')
absoluteImage5 = trueReforestationPotential.multiply(rangelandPotentialCover.divide(potentialCoverSum)).rename('RangelandPotential')
absoluteImage6 = trueReforestationPotential.multiply(pasturePotentialCover.divide(potentialCoverSum)).rename('PasturePotential')
absoluteImage7 = trueReforestationPotential.multiply(croplandPotentialCover.divide(potentialCoverSum)).rename('CroplandPotential')
absoluteImage8 = trueReforestationPotential.multiply(urbanPotentialCover.divide(potentialCoverSum)).rename('UrbanPotential')

## Calculate the potential numbers and write into local folder

In [10]:
# Stack the absolute biomass layers into an Image.
absPotentialImage = absoluteImage1.addBands(absoluteImage2).addBands(absoluteImage3).addBands(absoluteImage4).addBands(absoluteImage5).addBands(absoluteImage6).addBands(absoluteImage7).addBands(absoluteImage8).addBands(freelandForConservation)
# define the function to do the biome level statistics which could be applied by map      
def biomeLevelStat(biome):
    biomeMask = biomeLayer.eq(ee.Number(biome))
    masked_img = absPotentialImage.mask(biomeMask)
    output = masked_img.reduceRegion(reducer= ee.Reducer.sum(),
                                     geometry= unboundedGeo,
                                     crs='EPSG:4326',
                                     crsTransform=[0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                     maxPixels= 1e13)
    return output#.getInfo().get('Present')


biomeList = ee.List([1,2,3,4,5,6,7,8,9,10,11,12,13,14])
statisticTable = biomeList.map(biomeLevelStat).getInfo()
# transform into data frame
outputTable = pd.DataFrame(statisticTable,columns =['Present','PresentPotential','AbsolutePotential','FreelandPotential','RangelandPotential','PasturePotential','CroplandPotential','UrbanPotential','FreeToConservation'])#.round(1)
# sum the biome level biomass values to global total numbers
outputTable.loc['sum'] = outputTable.sum() 
outputTable.to_csv('Data/BiomeLevelStatistics/StatisticsForModels/AGB_HM1_Biome_Level_Statistics.csv',header=True,mode='w+')
# display the output of the carbon partitioning
print(colored('The biomass partition results in biome: \n', 'blue', attrs=['bold']))
outputTable.head(15)

The biomass partition results in biome: 



,Present,PresentPotential,AbsolutePotential,FreelandPotential,RangelandPotential,PasturePotential,CroplandPotential,UrbanPotential,FreeToConservation
0,136.050823,159.397390,174.439822,2.925035,0.044392,5.139731,6.736792,0.196483,4.747724
1,5.167177,6.855961,11.821483,1.262582,0.012142,1.184495,2.473694,0.032608,0.495146
2,1.743408,2.716664,3.471471,0.276040,0.019174,0.275060,0.182040,0.002493,0.282560
3,34.577677,41.638879,56.156988,3.408742,0.018042,4.233273,6.463535,0.394518,2.569602
4,16.351317,18.896847,21.169218,1.050201,0.030344,0.681533,0.473573,0.036720,1.295010
5,36.353804,44.557821,47.468668,2.572201,0.000767,0.173998,0.152073,0.011808,3.507886
6,31.511599,41.214407,60.746552,3.989852,6.948240,5.016059,3.537552,0.040442,2.822660
7,3.826574,4.549323,17.380046,2.369817,4.125838,1.677863,4.584007,0.073198,0.180862
8,1.206956,1.708240,2.457316,0.232091,0.262299,0.142758,0.105168,0.006760,0.093557
9,2.661718,3.327724,6.161922,0.810084,1.201062,0.422092,0.396482,0.004478,0.147619


In [14]:
# If you got the error 'EEException: Too many concurrent aggregations.', please re-run this chunck of code again.